# Scrape Jobindex

## Import packages

In [2]:
# import packages
import requests, json
import pandas as pd, numpy as np
import time, os

## Get web data

Procedure
collect the first page
save the totalresultcount for later usage


In [3]:
# the basic url
# https://www.jobindex.dk/jobsoegning.json?q=&jobage=archive&mindate=20180524&maxdate=20180824&page=2
url = 'https://www.jobindex.dk/jobsoegning.json'


# the request session
session = requests.session()
session.headers['email'] = 'pot@potweb.dk' 
session.headers['name'] = 'Peter Ottosen'
#session.headers['User-Agent'] = '' # sometimes you need to pose as another agent...
session.headers

# container for jobpostings
jobpostings = []
done = []

global last_t
last_t = time.time()

# function to create the next link
def create_link(offset_) :
    mindate = 20070101
    maxdate = 20070630
    addit = '?q=&jobage=archive&mindate=' + str(mindate) + '&maxdate=' + str(maxdate) + '&page=' + str(offset_)
    return url + addit

def ratelimit():
    time.sleep(1) # sleep one second.

# Reliable requests
def get(url,iterations=10,check_function=lambda x: x.ok):
    """This module ensures that your script does not crash from connection errors.
        that you limit the rate of your calls
        that you have some reliability check
        iterations : Define number of iterations before giving up. 
        exceptions: Define which exceptions you accept, default is all. 
    """
    for iteration in range(iterations):
        try:
            # add ratelimit function call here
            ratelimit() # !!
            response = session.get(url)
            if response.ok:
                return response # if succesful it will end the iterations here
        except exceptions as e: #  find exceptions in the request library requests.exceptions
            print(e) # print or log the exception message.
    return None # your code will purposely crash if you don't create a check function later.

def log_function(url,response,error_check=lambda x: x.ok,separator=',') :
    global last_t
#    logfilepath = r"C:\Users\pot\Documents\GitHub\group19\Eksamensprojekt/log_jobindex3a.csv"
    logfilepath = r"C:\notebooks\group19\Eksamensprojekt/log_jobindex3a.csv"
    if os.path.isfile(logfilepath) :
        f_log = open(logfilepath,'a')
    else:
        f_log = open(logfilepath,'w') # define logfile, remember not to overwrite it.
        # write columns to be used, basic ones are, servertime, deltaT since last call, url, success of request, 
        header = ['serverTime','deltaT','url','success','length','path']
        f_log.write(','.join(header)+'\n')
    #### Update timing info ####
    t = time.time()
    delta_t = t-last_t # calculate time since last call
    last_t = t# update last call time
    #### meta data ### 
    success = error_check(response)
    if success: # if call is successfull we add it to the done container
        done.append(url)
    if response.ok:
        length = len(response.text)
    else:
        length = 0
    row = [t,delta_t,url,success,length,path]
    f_log.write(separator.join(map(str,row))+'\n')
    f_log.close()


# just to get the number of results
#offset = 1
count = 0
#r = get(create_link(offset))
#d=json.loads(r.text)

#ResultCount=d['TotalResultCount']
#jp_keys = d['JobPositionPostings'][0].keys()

for pages in range(1, 19030) : # pagineringen
    offset = pages
    link = create_link(offset)
    if link in done: #check if you have already downloaded the link
        continue
    count +=1
    r = get(link)
    d=json.loads(r.text)
#    jobpostings.append(d['JobPositionPostings'])

    #define path 
#    base_path = r"C:\Users\pot\Documents\GitHub/jobindex3.txt"
    base_path = r"C:\notebooks/jobindex3a.txt"
    path = base_path      # use the link count as a filename
    if r.ok:
        if os.path.isfile(path) :
            f = open(path, mode="a", encoding="utf8")
        else:
            f = open(path, mode="w", encoding="utf8") # define logfile, remember not to overwrite it.
            
        html = r.text
        try :
            f.write(html)
            f.write('\n')
        except UnicodeEncodeError :
            print("Der opstod en UnicodeEncodeError ved skrivning af side", str(pages))
            f.write(html)
            f.write('\n')
        else :
            pass
        f.close()
    # run your log function
    log_function(link, r)

